In [1]:
import glob
import pickle
import pandas as pd
import numpy as np
import scipy
import nltk
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer
import random
from sklearn.decomposition import LatentDirichletAllocation 
from gensim import corpora, models, similarities
import jieba
import re

In [169]:
dfs = glob.glob('data/dataframes/*.p')
frames = []
for cdf in dfs:
    frames.append(pd.read_pickle(cdf))
df = pd.concat(frames)
df = df[df.is_student == True]
dataList = df["text"].tolist()
subjectList = df["subject"].tolist()
z = list(zip(dataList, subjectList))

random.shuffle(z)
dataList[:], subjectList[:] = zip(*z)

In [176]:
texts = dataList
texts = [jieba.lcut(text) for text in texts]
dictionary = corpora.Dictionary(texts)
feature_cnt = len(dictionary.token2id)
corpus = [dictionary.doc2bow(text) for text in texts]
tfidf = models.TfidfModel(corpus) 

subjects = subjectList
subjects = [jieba.lcut(subject) for subject in subjects]
subDict = corpora.Dictionary(subjects)
sub_fc = len(subDict.token2id)
subCorpus = [subDict.doc2bow(subject) for subject in subjects]

In [189]:
# print("Title:")
# subKw = input()
# print("Post:")
# keyword = input()
keyword = "I followed the instructions for adding the junit tests but I am still getting an error. Does anyone know what I'm doing wrong?"
subKw = "Junit test not working"

kw_vector = dictionary.doc2bow(jieba.lcut(keyword))
index = similarities.SparseMatrixSimilarity(tfidf[corpus], num_features = feature_cnt)
sim = index[tfidf[kw_vector]]

subTfidf = models.TfidfModel(subCorpus)
subKw_vector = subDict.doc2bow(jieba.lcut(subKw))
subIndex = similarities.SparseMatrixSimilarity(subTfidf[subCorpus], num_features = sub_fc)
subSim = subIndex[subTfidf[subKw_vector]]     

equal = 0
equal_id = 0
sub_bias = 0
sub_bias_id = 0
post_bias = 0
post_bias_id = 0

print("These posts might be similar to what you're looking for: \n")

for i in range(len(sim)):
    if sim[i]*1+subSim[i]*0.5 > equal:
        equal = sim[i]*1+subSim[i]*0.5
        equal_id = i

print('keyword is similar to text%d: %.2f' % (i, equal))
print(dataList[equal_id])
print("\n")

for i in range(len(sim)):
    if sim[i]*1+subSim[i]*1 > sub_bias:
        sub_bias = sim[i]*1+subSim[i]*1
        sub_bias_id = i   

print('keyword is similar to text%d: %.2f' % (i, sub_bias))
print(dataList[sub_bias_id])
print("\n")
 
for i in range(len(sim)):
    if sim[i]*1+subSim[i]*0 > post_bias:
        post_bias = sim[i]*1+subSim[i]*0.1
        post_bias_id = i        
        
print('keyword is similar to text%d: %.2f' % (i, post_bias))
print(dataList[post_bias_id])
print("\n")

These posts might be similar to what you're looking for: 

keyword is similar to text2176: 0.49
 I followed the a3 grader steps to add the Junit tests to work for a4, and a4 is in the class path, but the line that imports a4 is giving me an error. How do I fix this?      Edit: This is the error message that pops up. I have tried hovering over and manually clicking &#34;fix project setup&#34; as well.       Screen_Shot_20190928_at_2.48.20_PM.png  


keyword is similar to text2176: 0.89
 I followed the a3 grader steps to add the Junit tests to work for a4, and a4 is in the class path, but the line that imports a4 is giving me an error. How do I fix this?      Edit: This is the error message that pops up. I have tried hovering over and manually clicking &#34;fix project setup&#34; as well.       Screen_Shot_20190928_at_2.48.20_PM.png  


keyword is similar to text2176: 0.27
 PA.load("/dashboard/get_familiar", null, function(data){ $('#' + 'questionText').html(data);}); 


